## Handling of results from a single run of the simulation

In [1]:
# Import libraries
import pandas as pd
import json
import os 
import subprocess
import sys


In [18]:
# The settings
model = "3.5"
temperature = 1
initial_examples_generated = 100
feedback_iterations = 0
reboot = 5
specifications = "all"
output_folder = f"{model}-{initial_examples_generated}-{feedback_iterations}-{temperature}-{specifications}-{specifications}"

# Print the command to generate the code
print(f"python3 main.py generate_code_folder -d ../no_loop_problems/ -ieg {initial_examples_generated} -iter {feedback_iterations} -temp {temperature}  -reboot {reboot} -wpt 5 -o ../output/{output_folder} -output-file generated_code.c")


python3 main.py generate_code_folder -d ../no_loop_problems/ -ieg 100 -iter 0 -temp 1  -reboot 5 -wpt 5 -o ../output/3.5-100-0-1-all-all -output-file generated_code.c


In [28]:
# print all folders in the directory sorted by name
folders = os.listdir('../no_loop_problems/')

# Sort on number
folders.sort(key=lambda x: int(x.split('-')[0]))

# Create an empty dataframe that will be filled with info of the iterations
iteration_array = []

for folder_name in folders:
    # Read the output given in the file 
    with open(f"../output/{output_folder}/{folder_name}/results.txt", 'r') as file:
        # Read the file which contains an array
        data = json.load(file)
        
    verified_goals = []
    passed_tests = []

    # for each iteration in the array add it to the dataframe
    for i in data:
        # print the amount of passed tests
        verified_goals.append(i['verified_goals'])
        passed_tests.append(f"{i['test_information'][-1]['summary']['passed']} / {i['test_information'][-1]['summary']['total']}")

    # Add it as a column to the dataframe
    iteration_array.append(verified_goals)
    iteration_array.append(passed_tests)

# Create a dataframe with the information of the iterations
df = pd.DataFrame(iteration_array)

# export it to an excel file
df.to_clipboard(index=False, header=False)
    

## Work together with google sheets and google drive to store the information and results


In [10]:
# Import libraries
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

In [23]:
# get the instance of the Spreadsheet

# define the scope
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

# add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name('../tmp/vecogen-1ec545f23e03.json', scope)

# authorize the clientsheet 
client = gspread.authorize(creds)


# get the instance of the Spreadsheet
sheet = client.open('Master Thesis Merlijn Results')

# get the first sheet of the Spreadsheet
template_sheet_name = "template-model-initial_examples_generated-feedback_iterations-temperature-dataset"
template_sheet = sheet.worksheet(template_sheet_name)

In [37]:
# Copy the sheet called "template" to a new sheet
template_sheet_name = "template-model-initial_examples_generated-feedback_iterations-temperature-dataset"
new_sheet_name = f"{model}-{initial_examples_generated}-{feedback_iterations}-{temperature}-{specifications}-{specifications}"

# Copy the template sheet to a new sheet
result_sheet = sheet.duplicate_sheet(template_sheet.id)
result_sheet.update_title(new_sheet_name)

# Change the order to the last sheet
sheet.batch_update({
    "requests": {
        "updateSheetProperties": {
            "properties": {
                "sheetId": result_sheet.id,
                "index": len(sheet.worksheets()) - 1  # Move to the last position
            },
            "fields": "index"
        }
    }
})

# Add the data to the sheet
worksheet = sheet.worksheet(new_sheet_name)

# Add the data to the sheet
start_cell = 'E10'
end_cell =  'ZZZ69'

print(start_cell, end_cell)
# Convert DataFrame to list of lists
data_to_update = df.values.tolist()

# Update the range with the DataFrame values
worksheet.update(start_cell + ':' + end_cell, data_to_update)

E10 E110


C:\Users\MAL Sevenhuijsen\AppData\Local\Temp\ipykernel_5576\2774811292.py:34: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(start_cell + ':' + end_cell, data_to_update)


APIError: APIError: [400]: Requested writing within range ['3.5-100-0-1-all-all'!E10:E110], but tried writing to column [F]